## Cargar y explorar el dataset

**Instrucciones:**
- Descarga el dataset desde el repositorio de GitHub del curso. El archivo se encuentra en `datasets/pima_indian_diabetes_dataset/cleaned_dataset.csv`.
- Carga el dataset utilizando pandas.
- Muestra las primeras filas (`df.head()`) del dataset.
- Imprime la cantidad total de filas y columnas del dataset.

In [1]:
import pandas as pd
from IPython.display import display

# Cargar el dataset
df = pd.read_csv('../Asignacion_uno/cleaned_dataset.csv')

# Mostrar las primeras filas
print(df.head())

# Mostrar la cantidad total de filas y columnas
display(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

   Pregnancies  Glucose  Blood Pressure  Skin Thickness  Insulin   BMI  \
0            0      129             110              46      130  67.1   
1            0      180              78              63       14  59.4   
2            3      123             100              35      240  57.3   
3            1       88              30              42       99  55.0   
4            0      162              76              56      100  53.2   

   Diabetes Pedigree Function  Age  Outcome  
0                       0.319   26        1  
1                       2.420   25        1  
2                       0.880   22        0  
3                       0.496   26        1  
4                       0.759   25        1  


'Filas: 392, Columnas: 9'

## Crear función para cargar y dividir el dataset

**Instrucciones:**
- Implementa una función en Python que:
  - Cargue el dataset limpio desde la ruta especificada.
  - Seleccione las primeras 10 muestras como conjunto de entrenamiento.
  - Seleccione las siguientes 10 muestras como conjunto de prueba.
  - Devuelva por separado: `X_train`, `y_train`, `X_test`, `y_test`.

In [2]:
import numpy as np

# Tu código aquí
def cargar_y_dividir_dataset(df):
    # Seleccionar las primeras 10 muestras para entrenamiento
    train = df.iloc[:10]
    # Seleccionar las siguientes 10 muestras para prueba
    test = df.iloc[10:20]
    # Separar características y etiquetas
    X_train = train.drop('Outcome', axis=1).values
    y_train = train['Outcome'].values
    X_test = test.drop('Outcome', axis=1).values
    y_test = test['Outcome'].values
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = cargar_y_dividir_dataset(df)



## Comparar con scikit-learn

**Instrucciones:**
- Utiliza `KNeighborsClassifier` de scikit-learn para entrenar y predecir sobre el mismo subconjunto de datos. Asegúrate de definir los hiperparámetros: `k=3`, distancia euclidiana y método de búsqueda fuerza bruta (`algorithm='brute'`).
- Compara los resultados de tu implementación con los obtenidos por scikit-learn.
- El script debe mostrar una tabla que compare el valor real de `Resultado` de cada muestra de prueba, el valor predicho por tu algoritmo y el valor predicho por scikit-learn.

In [4]:
# Implementación KNN por fuerza bruta (distancia euclidiana, k=3) y comparación con sklearn
def knn_brute_predict(X_train, y_train, X_test, k=3):
    # distancias (num_test, num_train)
    dists = np.linalg.norm(X_test[:, None, :] - X_train[None, :, :], axis=2)
    neigh_idx = np.argsort(dists, axis=1)[:, :k]
    neigh_labels = y_train[neigh_idx]
    max_label = int(y_train.max()) + 1
    # voto por mayoría
    y_pred = np.array([np.bincount(row.astype(int), minlength=max_label).argmax() for row in neigh_labels])
    return y_pred

y_pred_custom = knn_brute_predict(X_train, y_train, X_test, k=3)

# Asegurar que y_pred_sklearn está definido: intentar usar knn_sklearn existente, si no, crear y entrenar
try:
    y_pred_sklearn
except NameError:
    try:
        y_pred_sklearn = knn_sklearn.predict(X_test)
    except Exception:
        knn_sklearn = KNeighborsClassifier(n_neighbors=3, algorithm='brute', metric='euclidean')
        knn_sklearn.fit(X_train, y_train)
        y_pred_sklearn = knn_sklearn.predict(X_test)

# Construir tabla comparativa
df_comparacion = pd.DataFrame({
    'y_true': y_test,
    'y_pred_custom': y_pred_custom,
    'y_pred_sklearn': y_pred_sklearn
})
df_comparacion['custom_ok'] = df_comparacion['y_true'] == df_comparacion['y_pred_custom']
df_comparacion['sklearn_ok'] = df_comparacion['y_true'] == df_comparacion['y_pred_sklearn']

# Mostrar resultados y métricas
display(df_comparacion)
acc_custom = df_comparacion['custom_ok'].mean()
acc_sklearn = df_comparacion['sklearn_ok'].mean()
print(f"Accuracy (custom k=3, euclidean, brute): {acc_custom:.2f}")
print(f"Accuracy (scikit-learn k=3, euclidean, brute): {acc_sklearn:.2f}")

,y_true,y_pred_custom,y_pred_sklearn,custom_ok,sklearn_ok
0,0,0,0,True,True
1,0,0,0,True,True
2,1,0,0,False,False
3,0,1,1,False,False
4,1,1,1,True,True
5,1,0,0,False,False
6,1,0,0,False,False
7,1,1,1,True,True
8,1,0,0,False,False
9,0,1,1,False,False


Accuracy (custom k=3, euclidean, brute): 0.40
Accuracy (scikit-learn k=3, euclidean, brute): 0.40
